# Install additional Python libraries

In [1]:
# !pip install quantities

# Delete existing DB

In [2]:
!rm simoc_server/db.sqlite

# Import Python libraries

In [3]:
import json
from random import randint

In [4]:
import pprint
pp = pprint.PrettyPrinter()

In [5]:
from simoc_server import app, db
from simoc_server.database.db_model import User
from simoc_server.database.seed_data import seed
from simoc_server.game_runner import GameRunnerManager, GameRunnerInitializationParams

Unknown DB_TYPE variable: "None"
Using SQLite by default


In [6]:
from simoc_server.views import convert_configuration

# Load Agent model

In [7]:
db.drop_all()
db.create_all()
seed.seed('agent_desc.json')

# Create new user

In [8]:
username = 'user1'
password = 'user1'

user = User.query.filter_by(username=username).first()
if not user:
    print('Creating new user')
    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()

Creating new user


# Initialize the ABM

In [9]:
game_config = {"agents": {
    "human_agent": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "food_storage": [1]}, "amount": 10}],
    "cabbage": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "lettuce": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "greenhouse_medium": [{"connections": {"power_storage": [1]}, "amount": 1}],
    "solar_pv_array": [{"connections": {"power_storage": [1]}, "amount": 100}],
    "multifiltration_purifier_post_treament": [{"connections": {"water_storage": [1, 2]}, "amount": 1}],
    "urine_recycling_processor_VCD": [{"connections": {"power_storage": [1], "water_storage": [1, 2]}, "amount": 1}],
    "co2_removal_SAWD": [{"connections": {"power_storage": [1], "air_storage": [1]}, "amount": 1}]},
"storages": {
    "air_storage": [{"id": 1, "atmo_h2o": 100, "atmo_o2": 100, "atmo_co2": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 0}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 1000}]},
"termination": {
    "time": {"value": 2000, "unit": "hour"}},
    "food_leaf": {"value": 10000, "unit": "kg"},
    "evacuation": {}
}

In [10]:
start_data = {"game_config": {'duration': {'value': 30, 'type': 'day'}, 'human_agent': {'amount': 4}, 'habitat': 'crew_habitat_small', 'greenhouse': 'greenhouse_small', 'food_storage': {'amount': 1000}, 'solar_arrays': {'amount': 60}, 'power_storage': {'amount': 30}, 'plants': [{'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 4}, {'species': 'rice', 'amount': 100}, {'species': 'wheat', 'amount': 30}, {'species': 'cabbage', 'amount': 3}]}}
game_config = convert_configuration(start_data["game_config"])
pp.pprint(game_config)

{'agents': {'cabbage': [{'amount': 3,
                         'connections': {'air_storage': [1],
                                         'food_storage': [1],
                                         'nutrient_storage': [1],
                                         'power_storage': [1],
                                         'water_storage': [1, 2]}}],
            'co2_reduction_sabatier': [{'amount': 1,
                                        'connections': {'air_storage': [1],
                                                        'power_storage': [1],
                                                        'water_storage': [1,
                                                                          2]}}],
            'co2_removal_SAWD': [{'amount': 1,
                                  'connections': {'air_storage': [1],
                                                  'power_storage': [1]}}],
            'crew_habitat_small': [{'amount': 1,
                                   

In [11]:
game_runner_manager = GameRunnerManager()
game_runner_init_params = GameRunnerInitializationParams(game_config)
game_runner_manager.new_game(user, game_runner_init_params)
game_runner = game_runner_manager.get_game_runner(user)

# Run the simiulation

In [12]:
num_steps = 100
for i in range(1, num_steps + 1):
    agent_model_state = game_runner_manager.get_step(user, i)
#     if i % 100 == 0:
#         print("Step: {}".format(i))
#         pp.pprint(agent_model_state)
pp.pprint(agent_model_state)

{'agents': {'total_agent_types': {'cabbage': 3,
                                  'co2_reduction_sabatier': 1,
                                  'co2_removal_SAWD': 1,
                                  'crew_habitat_small': 1,
                                  'greenhouse_small': 1,
                                  'human_agent': 4,
                                  'multifiltration_purifier_post_treament': 1,
                                  'oxygen_generation_SFWE': 1,
                                  'rice': 100,
                                  'solar_pv_array_mars': 60,
                                  'solid_waste_aerobic_bioreactor': 1,
                                  'urine_recycling_processor_VCD': 1,
                                  'wheat': 30},
            'total_consumption': {'atmo_co2': {'units': 'kg',
                                               'value': '0.2381'},
                                  'atmo_h2': {'units': 'kg', 'value': '0.0016'},
               

In [13]:
game_runner_manager.save_all(allow_repeat_save=False)

In [14]:
game_runner.agent_model.get_total_storages()

[{'agent_type': 'water_storage',
  'agent_id': 1,
  'currencies': [{'name': 'h2o_treated',
    'value': '0.0000',
    'capacity': 4000,
    'units': 'kg'},
   {'name': 'h2o_potb', 'value': '3998.8471', 'capacity': 4000, 'units': 'kg'},
   {'name': 'h2o_urin', 'value': '0.8413', 'capacity': 4000, 'units': 'kg'},
   {'name': 'h2o_wste', 'value': '0.0990', 'capacity': 4000, 'units': 'kg'},
   {'name': 'h2o_tret',
    'value': '783.2850',
    'capacity': 4000,
    'units': 'kg'}]},
 {'agent_type': 'food_storage',
  'agent_id': 1,
  'currencies': [{'name': 'food_edbl',
    'value': '956.5090',
    'capacity': 10000,
    'units': 'kg'}]},
 {'agent_type': 'water_storage',
  'agent_id': 2,
  'currencies': [{'name': 'h2o_treated',
    'value': '0.0000',
    'capacity': 4000,
    'units': 'kg'},
   {'name': 'h2o_potb', 'value': '3998.8471', 'capacity': 4000, 'units': 'kg'},
   {'name': 'h2o_urin', 'value': '94.8200', 'capacity': 4000, 'units': 'kg'},
   {'name': 'h2o_wste', 'value': '32.9300', '

In [15]:
game_runner.agent_model.get_total_agents()

{'total_production': {'atmo_o2': {'value': '535.4192', 'units': 'g'},
  'food_edbl': {'value': '0.0000', 'units': 'g'},
  'enrg_kwh': {'value': '31.8000', 'units': 'kWh'},
  'atmo_co2': {'value': '0.2575', 'units': 'kg'},
  'atmo_h2o': {'value': '0.4440', 'units': 'kg'},
  'h2o_wste': {'value': '0.2697', 'units': 'kg'},
  'h2o_urin': {'value': '1.7450', 'units': 'kg'},
  'atmo_ch4': {'value': '0.0033', 'units': 'kg'},
  'h2o_potb': {'value': '4.7500', 'units': 'kg'},
  'h2o_tret': {'value': '1.3600', 'units': 'kg'},
  'sold_n': {'value': '0.0022', 'units': 'kg'},
  'sold_p': {'value': '0.0003', 'units': 'kg'},
  'sold_k': {'value': '0.0006', 'units': 'kg'},
  'atmo_h2': {'value': '0.0454', 'units': 'kg'}},
 'total_consumption': {'atmo_co2': {'value': '0.2381', 'units': 'kg'},
  'h2o_potb': {'value': '3.2413', 'units': 'kg'},
  'sold_n': {'value': '0.0031', 'units': 'kg'},
  'sold_p': {'value': '0.0001', 'units': 'kg'},
  'sold_k': {'value': '0.0004', 'units': 'kg'},
  'enrg_kwh': {'val

In [16]:
game_runner.agent_model.get_model_stats()

{'step': 109,
 'hours_per_step': 1.0,
 'is_terminated': False,
 'time': 392400.0,
 'agents': {'total_production': {'atmo_o2': {'value': '535.4192',
    'units': 'g'},
   'food_edbl': {'value': '0.0000', 'units': 'g'},
   'enrg_kwh': {'value': '31.8000', 'units': 'kWh'},
   'atmo_co2': {'value': '0.2575', 'units': 'kg'},
   'atmo_h2o': {'value': '0.4440', 'units': 'kg'},
   'h2o_wste': {'value': '0.2697', 'units': 'kg'},
   'h2o_urin': {'value': '1.7450', 'units': 'kg'},
   'atmo_ch4': {'value': '0.0033', 'units': 'kg'},
   'h2o_potb': {'value': '4.7500', 'units': 'kg'},
   'h2o_tret': {'value': '1.3600', 'units': 'kg'},
   'sold_n': {'value': '0.0022', 'units': 'kg'},
   'sold_p': {'value': '0.0003', 'units': 'kg'},
   'sold_k': {'value': '0.0006', 'units': 'kg'},
   'atmo_h2': {'value': '0.0454', 'units': 'kg'}},
  'total_consumption': {'atmo_co2': {'value': '0.2381', 'units': 'kg'},
   'h2o_potb': {'value': '3.2413', 'units': 'kg'},
   'sold_n': {'value': '0.0031', 'units': 'kg'},
  